In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-ms-en.tar.gz
# !tar -zxf train-ms-en.tar.gz
# !rm train-ms-en.tar.gz

In [2]:
# !pip3 install pyenchant

In [3]:
import enchant
d = enchant.Dict("en_US")

In [4]:
d.check("Hello")

True

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json

In [6]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [7]:
!ls -lha train

total 890M
drwxr-xr-x  2 ubuntu ubuntu 4.0K Jun  26  2020 .
drwxr-xr-x 30 ubuntu ubuntu 4.0K Jul   4 21:57 ..
-rw-r--r--  1 ubuntu ubuntu 471M Jun  27  2020 left.txt
-rw-r--r--  1 ubuntu ubuntu 420M Jun  27  2020 right.txt


In [8]:
with open('train/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train/right.txt') as fopen:
    right = fopen.read().split('\n')

In [9]:
left[7106], right[7106]

('He said it was ridiculous for Apandi to state that investigations conducted by law enforcement agencies here had revealed that there was no evidence to show 1MDB money had been misappropriated.',
 'He said it was ridiculous for Apandi to state that investigations conducted by law enforcement agencies here had revealed that there was no evidence to show 1MDB money had been misappropriated.')

In [10]:
len(left), len(right)

(3712555, 3712555)

In [11]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [12]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [13]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)

In [14]:
rev_rules_compound_normalizer = defaultdict(list)
for k, v in rules_compound_normalizer.items():
    rev_rules_compound_normalizer[v].append(k)

In [15]:
rev_rules_normalizer = defaultdict(list)
for k, v in rules_normalizer.items():
    rev_rules_normalizer[v].append(k)

In [16]:
rules_compound_normalizer_regex = (
    '(?:' + '|'.join(list(rev_rules_compound_normalizer.keys())) + ')'
)

In [17]:
def _replace_compound(string):
    results = re.findall(
        rules_compound_normalizer_regex, string, flags=re.IGNORECASE
    )
    for r in results:
        string = string.replace(r, random.choice(rev_rules_compound_normalizer[r.lower()]))
    return string

In [18]:
_replace_compound('sudah tentu tidak')

'of cos tidak'

In [19]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = rev_rules_normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [20]:
string = 'saya sangat cantik"'
[replace_shortword(word) for word in string.split()]

['ana', 'bagaatt', 'cuaaantikk"']

In [21]:
[replace_shortword(word) for word in left[1].split()]

['"Pembangkang',
 'hnya',
 'brani',
 'cakap',
 'big,',
 'seolah-olah',
 'indah',
 'rupa',
 'drpada',
 'brita,"',
 'ktny.']

In [22]:
[replace_shortword(word, rules = en_lexicon) for word in 'do you like chicken'.split()]

['do', 'uuu', 'likee', 'chicken']

In [23]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [24]:
# random replace alignment
# random replace compound
# random replace word

In [25]:
random.random()

0.36992398912618685

In [26]:
malaya.augmentation.socialmedia_form('Saya')

['saye', 'sayak']

In [27]:
malaya.augmentation.vowel_alternate('saya')

'sya'

In [28]:
d.check("Hello")

True

In [46]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, p_replace_shortword = 0.7, p_socialmedia = 0.85, p_vowel_alternate = 0.5, p_replace_en_shortword = 0.3):
    
    left = _replace_compound(left)
    left = [replace_shortword(word, rules = en_lexicon) if random.random() > p_replace_en_shortword else word for word in left.split()]
    left = [(replace_shortword(word), False) if random.random() > p_replace_shortword else (word, True) for word in left]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        if d.check(l[0]):
            left_.append(l[0])
            continue
        
        if l[1]:
            if random.random() > p_socialmedia:
                r = malaya.augmentation.socialmedia_form(l[0])
                if len(r):
                    s = random.choice(r)
                else:
                    s = l[0]
            else:
                s = l[0]
            if random.random() > p_vowel_alternate:
                s = malaya.augmentation.vowel_alternate(s)
                if random.random() and s[-1] == 'a' and s[-2] in consonants:
                    s = s[:-1] + 'e'
        else:
            s = l[0]
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)

In [47]:
augment(left[0])

'Terminal 1 KKIA dilengkapi kemudahan 64 kaunter daftar masuk, 12 aero brdg slain mampu menampung 3,200 penumpang deep stu time.'

In [31]:
fast_text = malaya.language_detection.fasttext()

In [32]:
fast_text.predict([left[7106], right[7106], 'saya suka'])

['eng', 'eng', 'malay']

In [33]:
augment(left[0])

'Terminal 1 KKIA dlengkapi kemudahan 64 kaunter daftar masuk, 12 aero brdg selain mampu menmpng 3,200 pnumpng dlam stu mase'

In [34]:
from tqdm import tqdm

def loop(rows):
    rows, _ = rows
    new_left, new_right = [], []
    for i in tqdm(range(len(rows))):
        left, right = rows[i][0], rows[i][1]
        if len(left.split()) > 100 or len(right.split()) > 100:
            continue
        langs = fast_text.predict([left, right])
        if langs[0] == 'eng':
            continue
        if langs[1] == 'malay':
            continue
        try:
            new_left_ = augment(left)
            if new_left_ != left:
                new_left.append(new_left_)
                new_right.append(right)
        except Exception as e:
            pass
    return [[new_left, new_right]]

In [48]:
loop((list(zip(left[:20], right[:20])),0))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 3089.61it/s]


[[['Terminal 1 KKIA dlngkapi kmudhn 64 kaunter daftar in, 12 aero bridge selain mmpu menampung 3,200 passengers dlm satk mase',
   '"Pembangkang hny brani cakap besar, seolah-olah indah rupe daripade brita," katanya.',
   'Salah atu fakta yang paling kot adalh org Arab brniagk tuk hamba abdi di Afrika, meningkatkan pnyebarn dan dominasi budaya Arab.',
   'Macam mn dengan JoonPyo?',
   'Bogazkoy, Amasya Bogazkoy, Amasya merupakan sbuah kawasan yang trltk d Turki.',
   'Beliau dianggp sebagai ahli falsafa yang paling agung pernah dilahirkan dlam sejarah Islam.',
   'Dan bacakanlah whai Muhammad) d dlam Kitab (Al-Quran) inik perihal Nabi Ibrahim; sesungguhnyak adalh ia seorang yng amat bner, lagi menjdi Nabi.',
   'Tersedia scare nasional mellui dua layanan stlt di negara itu Star Choice, pde saluran 276 dn Bell TV pda slurn 828.',
   'Pelapis baru pn dh so lma juga termasuk Kaj Teh.',
   'Pda masa nie, jmaah menteri Dr Mahathir hnya mempunyai 13 anggte sedangkan ada beberapa portfolio pe

In [36]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [37]:
import mp

In [49]:
r = mp.multiprocessing(list(zip(left[1300000:1800100], right[1300000:1800100])), loop, cores = 4)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 125025/125025 [00:43<00:00, 2905.66it/s]


In [50]:
en, ms = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    ms.extend(r[i][0])
    en.extend(r[i][1])

0 104981
1 104919
2 104857
3 104993


In [51]:
len(ms), len(en)

(419750, 419750)

In [52]:
ms[-10:]

['Beliau berkata, kbanyakn ngare mju sperti di Eropah, Amerika Utara dan Japan juga ngggk terlepas berdepan risiko rasuah dlam service air.',
 'Ia mngurs beberapak pertandngn yang paling terkenal adl Piala FA.',
 'Suwit berkata, hasil siasatan awal menunjukkan sekumpulan enam hngga tujuh penyerang menaiki dua kenderaan n memakai pakaian seragam tentera menyerbu rumh trbe 4 petang waktu temptn srte menahan ksemua penghninye',
 'Jelas Mohamad, pandangan dn jugak dasar yg dilemparkan ithu patut dikecam.',
 '"Maklumat skim ini bolh didapati dengan mudah melalui laman web myslm d ktny dalm kenytaan dhi Putrajaya hari ini',
 'tanpe benar bnr mengatakannya.',
 'Lapangan Terbang Antarabangsa Gen Heriberto Jara Lapangan Terbang Antarabangsa Gen Heriberto Jara adlh sebua lapangan terbng yang terletak di bandr Veracruz, Mexico.',
 'Filem laknan Kendrick paling berjaya pada 2012 ialah sbuah filem mzkl komedi "Pitch Perfect" (2012), yanckh berdasarkan scr longgar drii bku bukan cerekak "Pitch Perfe

In [53]:
en[-10:]

['He said most developed countries such as Europe, North America and Japan did not miss the risk of corruption in water services.',
 'It manages several competitions, most notably the FA Cup.',
 'Sutang said preliminary investigations showed a group of six to seven attackers boarding two vehicles and wearing military uniforms raided the house at 4pm local time and detained all of its occupants.',
 "It is clear that Mohamad's views and policies should be criticized.",
 '"Information on the scheme is readily available through the website MySalam, he said in a statement in Putrajaya today.',
 'without really saying it.',
 'Gen Heriberto Jara International Airport Gen Heriberto Jara International Airport is an airport located in the city of Veracruz, Mexico.',
 'Kendrick\'s most successful movie in 2012 was the comedy musical "Pitch Perfect" (2012), based loosely on the non-fiction book "Pitch Perfect: The Quest for Collegiate A Cappella Glory".',
 'There are 7 villages in the district of 

In [54]:
import json

with open('augmented-ms-en-v3.json', 'w') as fopen:
    json.dump({'en': en, 'ms': ms}, fopen)

In [55]:
with open('augmented-ms-en-v3.json') as fopen:
    data = json.load(fopen)